In [5]:
from pathlib import Path
import os

import pandas as pd
import numpy as np
from data_processing.data_processing import *

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [6]:
MAIN_PATH = Path(os.getcwd()).parent

## Load data

In [168]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))
org_df = load(org_df)
org_df

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,temp_zuz
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87,1297.0
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:03:00+00:00,54.154394,54.985713,12.052065,22.335388,108.623473,112.386061,109.836948,108.717994,110.464245,104.418573,...,92.024462,92.405153,24.661329,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:04:00+00:00,54.693184,54.490742,12.027310,22.412620,108.746784,112.396792,109.966910,108.978605,110.603660,104.464328,...,92.024590,92.405110,24.660003,24.525475,4.48,8.98,8.6,4.55,9.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 22:55:00+00:00,57.882366,58.080133,10.056829,30.486496,101.111263,98.338021,102.332506,101.967828,100.651891,97.071359,...,98.121880,98.225985,24.460338,24.320295,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:56:00+00:00,57.953225,57.931008,9.935828,29.830075,101.034501,98.362465,102.332450,101.981035,100.575884,97.066000,...,98.121319,98.226027,24.464324,24.322539,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:57:00+00:00,58.186232,57.291012,10.014864,29.350514,101.251020,98.353221,102.312023,102.017297,100.621084,97.223686,...,98.120757,98.226070,24.468310,24.324782,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:58:00+00:00,59.016838,58.014325,10.048125,29.496899,101.073264,98.354741,102.336123,101.967655,100.596174,97.050953,...,98.120196,98.226113,24.472296,24.327025,6.18,8.89,7.8,6.65,12.47,NaN


In [169]:
df = pd.DataFrame(index=org_df.index)

## Simple features based on correlation

In [170]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df['TIX1'] = org_df[TIX1].mean(axis='columns')

In [171]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df['TIR'] = org_df[TIR].mean(axis='columns')

In [172]:
# dodanie reg nadawy koncentratu
FCX = org_df.columns[org_df.columns.str.contains('fcx')].values
df[FCX] = org_df[FCX]

In [173]:
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df[NIR] = org_df[NIR]

In [174]:
# dodanie temp żużla
df["TEMP_ZUZ"] = org_df["temp_zuz"]

In [175]:
pr_df = df.copy()
prw_df = df.copy()
df = df.dropna(axis=0)
df.shape

(11377, 8)

In [176]:
df

,TIX1,TIR,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ
2020-09-30 22:00:00+00:00,418.990965,29.086390,56.729077,54.724422,11.966905,22.493207,14.362428,1297.0
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.309363,55.327746,11.544775,22.702118,14.978720,1295.0
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.703949,54.957514,11.053689,20.013150,15.199544,1303.0
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.000000,55.224717,10.980221,19.902115,14.815123,1302.0
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.886746,54.869892,11.057164,21.069357,14.617263,1303.0
...,...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,418.284737,27.694067,59.180029,58.900365,10.019018,32.147630,15.607105,1300.0
2022-01-31 19:00:00+00:00,418.138199,27.760225,58.895968,58.911688,10.013093,31.719701,14.846545,1305.0
2022-01-31 20:00:00+00:00,418.012402,27.761558,59.073501,59.593765,10.064700,30.158128,15.967954,1304.0
2022-01-31 21:00:00+00:00,417.939267,27.704117,57.774647,58.738514,9.989189,28.000376,16.147269,1308.0


## Split data

In [107]:
train, val, test = split(df)

In [108]:
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

## Ridge Regression

In [143]:
ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model.predict(X_val))

74.98678833682321

## Polynomial Regression

In [111]:
poly_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                      ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model = poly_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model.predict(X_val))

68.24177515684886

In [112]:
poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                      ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model2 = poly_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model2.predict(X_val))

70.73612696044367

In [114]:
poly_model3 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                        ('normalization', preprocessing.StandardScaler()),
                        ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model3 = poly_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, poly_model3.predict(X_val))

1697403.2739010963

## SGD Regressor

In [69]:
sgd_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])

In [70]:
sgd_model1 = sgd_model1.fit(X_train, y_train)

In [71]:
metrics.mean_squared_error(y_val, sgd_model1.predict(X_val))

74.99823953028356

In [72]:
sgd_model2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model2 = sgd_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model2.predict(X_val))

74.95110580046402

In [73]:
sgd_model3 = Pipeline([('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l1', learning_rate='adaptive'))])
sgd_model3 = sgd_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model3.predict(X_val))

7.702792273803667e+16

## SGD + Poly Regression

In [74]:
sgd_poly_model1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model1 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

67.59070040812387

In [75]:
sgd_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model2 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

70.15790790973689

In [76]:
sgd_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model2 = sgd_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model1.predict(X_val))

69.1222819715861

## Poly + Ridge Regression

In [122]:
ridge_poly_model1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='cholesky'))])
ridge_poly_model1 = ridge_poly_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model1.predict(X_val))

63.81028513099203

In [159]:
ridge_poly_model1.named_steps["ridge"].coef_

array([  0.        ,   2.05630841,  -3.07491127,  -5.27460674,
       -11.01781215,   1.69838657,   8.12500049,   7.66617289,
        -2.85566669,   2.99128159,   0.4373027 ,   1.85818547,
        -6.69476602,   1.1046304 ,   5.11141966,  -2.19087638,
        -3.93591315,  -7.20307755,   0.93748825,   2.26718124,
         4.90754517,  -5.28920633,  -1.38081289,  16.3833378 ,
         9.30270583,   1.21474811,  -5.69970881,  11.70779359,
        11.48920987,  -3.75548009,   7.30526562,   1.51733866,
        -1.2869995 ,   3.04514545,  -4.66722942,  -5.0443614 ,
        -8.39873166,   2.81038931,   7.18143668,  15.89236583,
       -13.3560872 ,  -4.90641582,   1.70823611,   2.43312497,
         5.35236304,   8.43174561,  -6.99898215,  -3.89352348,
         3.40809991, -12.08042451,   3.89533945,   6.00033915,
        -4.97989552,   2.64715968,  -4.56524318,   1.15978057,
         4.65436461,   0.50481481,  -1.17942426,   0.13209437,
         0.03301875,  -8.55528031,  -9.39068892,  -9.66

In [117]:
ridge_poly_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model2 = ridge_poly_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model2.predict(X_val))

67.30556340464635

In [118]:
ridge_poly_model3 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model3 = ridge_poly_model3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model3.predict(X_val))

65.55260735524168

In [120]:
ridge_poly_model4 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=5)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='saga'))])
ridge_poly_model4 = ridge_poly_model4.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model4.predict(X_val))

64.95187841501655

## Spline + Ridge Regression

In [134]:
ridge_spline_model1 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model1 = ridge_spline_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model1.predict(X_val))

63.66222365050161

In [142]:
ridge_spline_model2 = Pipeline([('spline', preprocessing.SplineTransformer(n_knots=3, knots='quantile')),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_spline_model2 = ridge_spline_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_spline_model2.predict(X_val))

65.23108527165992

## ElasticNet Regression

In [124]:
elastic_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=10000))])
elastic_model1 = elastic_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model1.predict(X_val))

79.05195742848755

In [129]:
elastic_model2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=4)),
                           ('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=10000))])
elastic_model2 = elastic_model2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model2.predict(X_val))

77.32094660157615

## Bayessian Regression

In [145]:
bay_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('bayessian', linear_model.BayesianRidge(n_iter=1000))])
bay_model = bay_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, bay_model.predict(X_val))

74.98947741123759

In [152]:
bay_poly_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3, interaction_only=True)),
                           ('normalization', preprocessing.StandardScaler()),
                            ('bayessian', linear_model.BayesianRidge(n_iter=1000, verbose=True, tol=1e-5))])
bay_poly_model = bay_poly_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, bay_poly_model.predict(X_val))

Convergence after  83  iterations


63.19306084127666

In [158]:
bay_poly_model.named_steps["bayessian"].coef_

array([ 4.80566373e-12, -1.27737842e+01, -3.66216690e+00,  1.99264333e+01,
       -2.96602016e+01,  4.58976625e+01,  1.02613197e+01,  3.76230780e+01,
        1.35429929e+01, -2.26877368e+01,  2.68332785e+01, -9.14027722e+01,
        5.60965667e+01,  1.75459064e+01, -3.86496152e+01, -7.69194078e+01,
        7.60224658e+01, -4.90230804e+01, -1.43473519e+01,  1.10271331e+01,
        5.15474491e+01,  4.11272544e+01,  1.90069365e+01,  7.12665600e+01,
        3.40646700e+01,  3.21812416e+01, -4.36377720e+01, -1.22902331e+02,
       -6.71744660e+01,  4.22756714e+01,  8.72615837e+01, -6.11779361e+01,
       -1.81737837e+01, -3.43348264e+01,  4.48304164e+01, -6.46353752e+01,
       -8.78267772e+01, -3.87895318e+01, -7.65060962e+01, -5.13692001e+01,
       -8.05718714e+01,  8.66941491e+01,  1.67057365e+02,  5.67537368e+01,
       -9.82888641e+01,  8.54527274e+01,  4.24270177e+01,  2.32884724e+01,
        7.55045309e+00,  4.73293769e+01,  3.32206428e+01, -6.33819104e+01,
       -2.57307564e+01,  

# Using all features, without average

### Split data

In [90]:
all_df = org_df.dropna()
all_df

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,temp_zuz
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.60,4.55,9.87,1297.0
2020-09-30 23:00:00+00:00,55.309363,55.327746,11.544775,22.702118,108.378922,112.237887,109.862477,109.128895,110.520359,104.410075,...,92.033540,92.402722,24.585718,24.448561,4.48,8.98,8.60,4.55,9.87,1295.0
2020-10-01 00:00:00+00:00,54.703949,54.957514,11.053689,20.013150,109.861116,112.517942,110.237388,109.214988,110.434876,104.657074,...,92.056562,92.400504,24.564115,24.484372,4.48,8.98,8.60,4.55,9.87,1303.0
2020-10-01 00:30:00+00:00,55.000000,55.224717,10.980221,19.902115,109.585113,112.805842,110.151395,108.958777,110.694621,104.680675,...,92.068073,92.403063,24.610243,24.517869,4.48,8.98,8.60,4.55,9.87,1302.0
2020-10-01 01:00:00+00:00,55.886746,54.869892,11.057164,21.069357,109.757104,112.602264,110.165194,109.178997,110.596014,104.695973,...,92.076607,92.405623,24.605002,24.488603,4.30,7.84,8.61,4.58,9.74,1303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,59.180029,58.900365,10.019018,32.147630,103.049852,98.548160,102.593229,102.306346,100.762990,97.234765,...,98.130381,98.257987,24.401280,24.226479,6.38,9.12,7.74,6.06,11.26,1300.0
2022-01-31 19:00:00+00:00,58.895968,58.911688,10.013093,31.719701,102.570421,98.549435,102.571353,102.108130,100.757045,97.183075,...,98.131138,98.234942,24.301374,24.145683,6.38,9.12,7.74,6.06,11.26,1305.0
2022-01-31 20:00:00+00:00,59.073501,59.593765,10.064700,30.158128,102.197750,98.577139,102.375469,102.194845,100.746604,97.138241,...,98.130569,98.218519,24.326187,24.222134,6.38,9.12,7.74,6.06,11.26,1304.0
2022-01-31 21:00:00+00:00,57.774647,58.738514,9.989189,28.000376,102.266821,98.477571,102.374577,102.107050,100.649014,97.129910,...,98.130687,98.221079,24.290438,24.144540,6.38,9.12,7.74,6.06,11.26,1308.0


In [91]:
train_a, val_a, test_a = split(all_df)

In [92]:
X_train_a, X_val_a, X_test_a = train_a.drop(["temp_zuz"], axis=1), val_a.drop(["temp_zuz"], axis=1), test_a.drop(["temp_zuz"], axis=1)
y_train_a, y_val_a, y_test_a = train_a["temp_zuz"], val_a["temp_zuz"], test_a["temp_zuz"]

### Polynomial Regression

In [93]:
poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                          ('linear', linear_model.LinearRegression(fit_intercept=False))])
poly_model_all = poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, poly_model_all.predict(X_val_a))

3668115.416767741

### Poly + SGD Regression

In [94]:
sgd_poly_model_all = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='elasticnet', learning_rate='adaptive'))])
sgd_poly_model_all = sgd_poly_model_all.fit(X_train_a, y_train_a)
metrics.mean_squared_error(y_val_a, sgd_poly_model_all.predict(X_val_a))

6.053012405180626e+21

Wniosek: zdecydowanie nie chcemy brać wszystkich feauterów

# Adding PR to features

In [160]:
# wrzucam o prażonkach i prob
PR = org_df.columns[org_df.columns.str.contains('pr')].values
pr_df[PR] = org_df[PR]

In [161]:
# split data
pr_df = pr_df.dropna(axis=0)
train, val, test = split(pr_df)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

X_train

In [163]:
y_train

2020-09-30 22:00:00+00:00    1297.0
2020-09-30 23:00:00+00:00    1295.0
2020-10-01 00:00:00+00:00    1303.0
2020-10-01 00:30:00+00:00    1302.0
2020-10-01 01:00:00+00:00    1303.0
                              ...  
2021-09-05 20:00:00+00:00    1306.0
2021-09-05 21:00:00+00:00    1303.0
2021-09-05 22:00:00+00:00    1301.0
2021-09-05 23:00:00+00:00    1304.0
2021-09-06 00:00:00+00:00    1306.0
Name: TEMP_ZUZ, Length: 7963, dtype: float64

### Poly + SGD Regression

In [84]:
sgd_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=2)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_poly_model_pr = sgd_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_poly_model_pr.predict(X_val))

566402038.8214126

In [164]:
sgd_poly_model_pr.named_steps["sgd"].coef_

array([ 0.00000000e+00, -5.80654818e+03,  3.07434180e+03,  9.86322418e+03,
       -2.16307564e+04, -7.01697444e+01, -1.58365893e+04, -8.15712936e+03,
       -7.98387634e+03, -7.65621816e+02,  1.69223360e+04, -1.83406217e+04,
        3.73863818e+03,  9.36372092e+02,  9.11560303e+03, -1.09026289e+02,
       -7.75076973e+03, -1.57565450e+03, -6.37547856e+03,  7.42347560e+03,
       -9.49543760e+03,  3.06919304e+03,  5.66262907e+03, -1.10226481e+04,
        4.05522179e+03, -7.09109463e+03,  6.40737195e+04, -5.78050497e+04,
        4.21629472e+03,  4.80280565e+04, -3.20081667e+04, -3.10269830e+03,
        1.24121283e+04, -3.80249956e+03, -5.53390875e+03, -8.76080897e+03,
       -3.35753139e+04, -3.16496657e+04, -1.62432005e+04, -9.81468484e+03,
        6.18371470e+04,  4.26234624e+04, -5.97521842e+04, -3.97976734e+04,
        2.69002394e+04, -2.08418262e+04, -2.79548228e+04,  7.89189178e+04,
        1.16925918e+04,  4.08677570e+04,  3.82114850e+04,  3.79329408e+04,
       -2.16947814e+03,  

### SGD Regression

In [165]:
sgd_model_pr = Pipeline([('normalization', preprocessing.StandardScaler()),
                       ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty='l2', learning_rate='adaptive'))])
sgd_model_pr = sgd_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, sgd_model_pr.predict(X_val))

78.12499659869678

In [166]:
sgd_model_pr.named_steps["sgd"].coef_

array([ 1.07951294,  0.37283907, -0.79076949,  1.68263232,  0.24696015,
        0.68066671,  0.60738379,  0.08032435, -0.06346587,  0.30691705,
        0.07899788,  0.52351158])

### Poly + Ridge Regression

In [99]:
ridge_poly_model_pr = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='sag'))])
ridge_poly_model_pr = ridge_poly_model_pr.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr.predict(X_val))

79.53811201031888

In [100]:
ridge_poly_model_pr1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_poly_model_pr1 = ridge_poly_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr1.predict(X_val))

77.6476214726276

In [103]:
ridge_poly_model_pr2 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='saga'))])
ridge_poly_model_pr2 = ridge_poly_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_pr2.predict(X_val))

79.4660965356888

### Ridge Regression

In [104]:
ridge_model_pr1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model_pr1 = ridge_model_pr1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr1.predict(X_val))

78.15592482417102

In [167]:
ridge_model_pr1.named_steps["ridge"].coef_

array([ 1.08209998,  0.3691561 , -0.7964524 ,  1.66777108,  0.25955482,
        0.68953828,  0.60649385,  0.07767323, -0.0602534 ,  0.30672109,
        0.08191634,  0.52413647])

In [105]:
ridge_model_pr2 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='sag'))])
ridge_model_pr2 = ridge_model_pr2.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr2.predict(X_val))

78.15381925985238

In [106]:
ridge_model_pr3 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='svd'))])
ridge_model_pr3 = ridge_model_pr3.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_pr3.predict(X_val))

78.15592482417102

# Add pr, wentylatory i moc cieplną

In [192]:
# wrzucam o prażonkach i prob
PR = org_df.columns[org_df.columns.str.contains('pr')].values
prw_df[PR] = org_df[PR]

In [193]:
# wentylatory
UXM = org_df.columns[org_df.columns.str.contains('uxm')].values
prw_df[UXM] = org_df[UXM]

In [194]:
# dodać moc cieplną
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
prw_df[NIR] = org_df[NIR]

In [195]:
# dodać temp na kol
TXI = org_df.columns[org_df.columns.str.contains('txi')].values
prw_df[TXI] = org_df[TXI]

In [196]:
prw_df

,TIX1,TIR,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,001txi01153.daca.pv,001txi01154.daca.pv
2020-09-30 22:00:00+00:00,418.990965,29.086390,56.729077,54.724422,11.966905,22.493207,14.362428,1297.0,4.48,8.98,8.60,4.55,9.87,92.090454,92.024078,92.405281,19.126427,19.929581
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.309363,55.327746,11.544775,22.702118,14.978720,1295.0,4.48,8.98,8.60,4.55,9.87,92.129309,92.033540,92.402722,18.885216,19.536727
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.703949,54.957514,11.053689,20.013150,15.199544,1303.0,4.48,8.98,8.60,4.55,9.87,92.104858,92.056562,92.400504,18.612499,19.254154
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.000000,55.224717,10.980221,19.902115,14.815123,1302.0,4.48,8.98,8.60,4.55,9.87,92.114715,92.068073,92.403063,18.455667,19.196964
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.886746,54.869892,11.057164,21.069357,14.617263,1303.0,4.30,7.84,8.61,4.58,9.74,92.091896,92.076607,92.405623,18.075895,18.785391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,418.284737,27.694067,59.180029,58.900365,10.019018,32.147630,15.607105,1300.0,6.38,9.12,7.74,6.06,11.26,98.037160,98.130381,98.257987,9.714017,10.275810
2022-01-31 19:00:00+00:00,418.138199,27.760225,58.895968,58.911688,10.013093,31.719701,14.846545,1305.0,6.38,9.12,7.74,6.06,11.26,98.057871,98.131138,98.234942,9.762848,10.447076
2022-01-31 20:00:00+00:00,418.012402,27.761558,59.073501,59.593765,10.064700,30.158128,15.967954,1304.0,6.38,9.12,7.74,6.06,11.26,98.065827,98.130569,98.218519,9.548869,10.267305
2022-01-31 21:00:00+00:00,417.939267,27.704117,57.774647,58.738514,9.989189,28.000376,16.147269,1308.0,6.38,9.12,7.74,6.06,11.26,98.016144,98.130687,98.221079,9.311281,9.778013


In [197]:
# split data
prw_df = prw_df.dropna(axis=0)
train, val, test = split(prw_df)
print(train.shape)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"], axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

(7963, 18)


In [199]:
X_train

,TIX1,TIR,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,001uxm0rf01.daca.pv,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,001txi01153.daca.pv,001txi01154.daca.pv
2020-09-30 22:00:00+00:00,418.990965,29.086390,56.729077,54.724422,11.966905,22.493207,14.362428,4.48,8.98,8.60,4.55,9.87,92.090454,92.024078,92.405281,19.126427,19.929581
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.309363,55.327746,11.544775,22.702118,14.978720,4.48,8.98,8.60,4.55,9.87,92.129309,92.033540,92.402722,18.885216,19.536727
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.703949,54.957514,11.053689,20.013150,15.199544,4.48,8.98,8.60,4.55,9.87,92.104858,92.056562,92.400504,18.612499,19.254154
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.000000,55.224717,10.980221,19.902115,14.815123,4.48,8.98,8.60,4.55,9.87,92.114715,92.068073,92.403063,18.455667,19.196964
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.886746,54.869892,11.057164,21.069357,14.617263,4.30,7.84,8.61,4.58,9.74,92.091896,92.076607,92.405623,18.075895,18.785391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-05 20:00:00+00:00,427.223464,27.474773,54.130574,54.970550,4.938091,24.345593,22.548988,4.11,4.74,8.48,5.54,11.24,97.187363,97.117664,97.511354,20.054366,20.598770
2021-09-05 21:00:00+00:00,426.885975,27.742631,55.248131,54.544512,4.930626,26.030218,23.326609,4.72,7.72,8.64,5.58,11.40,97.175505,97.125689,97.531042,19.182281,19.653056
2021-09-05 22:00:00+00:00,426.514867,27.705729,55.808769,55.667976,4.994519,25.914533,21.962930,4.72,7.72,8.64,5.58,11.40,97.171569,97.112901,97.528480,18.364518,18.825136
2021-09-05 23:00:00+00:00,426.142533,27.594741,57.056538,56.859436,4.999296,24.831633,21.440236,4.72,7.72,8.64,5.58,11.40,97.175270,97.101088,97.522388,17.437377,17.925498


In [200]:
ridge_model_prw1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=1, solver='cholesky'))])
ridge_model_prw1 = ridge_model_prw1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_model_prw1.predict(X_val))

72.64209302789492

In [201]:
ridge_model_prw1.named_steps["ridge"].coef_

array([-0.02124518,  0.64242082, -0.4945489 ,  1.92445126, -0.8135563 ,
        0.46787638,  0.32627174,  0.01486902, -0.23157003,  0.56478471,
        0.33195436,  0.31314978,  1.19699367,  0.23224349,  0.44038654,
       -2.93386004,  2.67356196])

In [202]:
ridge_poly_model_prw1 = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=3)),
                            ('normalization', preprocessing.StandardScaler()),
                            ('ridge', linear_model.Ridge(alpha=.5, solver='svd'))])
ridge_poly_model_prw1 = ridge_poly_model_prw1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, ridge_poly_model_prw1.predict(X_val))

103.035787769448

In [203]:
ridge_poly_model_prw1.named_steps["ridge"].coef_

array([-1.06893022e-12, -1.02296523e+01,  6.16181857e+00, ...,
        5.44102976e-01,  9.37271321e-01,  8.11542667e-01])